# 时间序列

## 日期和时间数据类型及工具

In [1]:
from datetime import datetime

In [3]:
now = datetime.now()

In [4]:
now

datetime.datetime(2017, 7, 14, 15, 45, 3, 163000)

In [5]:
now.year, now.month, now.day

(2017, 7, 14)

In [6]:
# datetime 以毫秒的形式存储日期和时间
# datetime.timedelta表示两个datetime对象之间的时间差：

In [7]:
delta = datetime(2011, 1, 7) - datetime(2008, 6, 24, 8, 15)

In [8]:
delta

datetime.timedelta(926, 56700)

In [10]:
delta.days

926

In [11]:
delta.seconds

56700

In [12]:
from datetime import timedelta

In [13]:
start = datetime(2011, 1, 7)

In [14]:
start + timedelta(12)

datetime.datetime(2011, 1, 19, 0, 0)

In [15]:
start - 2*timedelta(12)

datetime.datetime(2010, 12, 14, 0, 0)

### 字符串和datetime的相互转换

#### 利用str或strftime方法（传入一个格式化字符串），datetime对象hepandas的Timestamp对象可以被格式化为字符串：

In [16]:
stamp = datetime(2011, 1, 3)

In [17]:
str(stamp)

'2011-01-03 00:00:00'

In [18]:
stamp.strftime('%Y-%m-%d')

'2011-01-03'

In [19]:
value = '2011-01-03'

In [20]:
datetime.strptime(value, '%Y-%m-%d')

datetime.datetime(2011, 1, 3, 0, 0)

In [21]:
datestrs = ['7/6/2011', '8/6/2011']

In [23]:
[datetime.strptime(x, '%m/%d/%Y') for x in datestrs]

[datetime.datetime(2011, 7, 6, 0, 0), datetime.datetime(2011, 8, 6, 0, 0)]

#### datetime.striptime是通过已知格式进行日期解析的最佳方式，但每次都编写格式定义很麻烦
此时可以使用第三方包中的parser.parse方法

In [24]:
from dateutil.parser import parse

In [25]:
parse('2011-01-03')

datetime.datetime(2011, 1, 3, 0, 0)

In [27]:
parse('Jan 31, 1997, 10:45 PM')

datetime.datetime(1997, 1, 31, 22, 45)

#### 在国际通用格式中，日通常出现在月的前面，传入dayfirst=true可以解决这个问题

In [28]:
parse('6/12/2011', dayfirst=True)

datetime.datetime(2011, 12, 6, 0, 0)

In [30]:
datestrs

['7/6/2011', '8/6/2011']

In [33]:
from pandas import Series,DataFrame
import pandas as pd
%pylab inline
pd.to_datetime(datestrs)

Populating the interactive namespace from numpy and matplotlib


E:\Anaconda\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


DatetimeIndex(['2011-07-06', '2011-08-06'], dtype='datetime64[ns]', freq=None)

In [34]:
# 还可以处理缺失值
idx = pd.to_datetime(datestrs + [None])

In [35]:
idx

DatetimeIndex(['2011-07-06', '2011-08-06', 'NaT'], dtype='datetime64[ns]', freq=None)

In [36]:
idx[2]

NaT

In [37]:
pd.isnull(idx)

array([False, False,  True], dtype=bool)

#### NaT（Not a Time）是pandas中时间戳数据的NA值

### 时间序列基础

#### pandas最基本的时间序列类型就是以时间戳为索引的Series（通常以python字符串huodatatime对象表示）

In [39]:
from datetime import datetime
dates = [datetime(2011, 1, 2),datetime(2011, 1, 5), datetime(2011, 1, 7),
        datetime(2011, 1, 8), datetime(2011, 1, 10), datetime(2011, 1, 12)]

In [40]:
dates

[datetime.datetime(2011, 1, 2, 0, 0),
 datetime.datetime(2011, 1, 5, 0, 0),
 datetime.datetime(2011, 1, 7, 0, 0),
 datetime.datetime(2011, 1, 8, 0, 0),
 datetime.datetime(2011, 1, 10, 0, 0),
 datetime.datetime(2011, 1, 12, 0, 0)]

In [41]:
ts = Series(np.random.randn(6), index=dates)

In [42]:
ts

2011-01-02    1.874317
2011-01-05   -0.148317
2011-01-07    0.414202
2011-01-08    0.015564
2011-01-10    1.650215
2011-01-12    0.248404
dtype: float64

In [43]:
type(ts)

pandas.core.series.Series

In [44]:
ts.index

DatetimeIndex(['2011-01-02', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)

#### 不同索引的时间序列之间的算数运算会自动按日期对齐

In [45]:
ts + ts[::2]

2011-01-02    3.748635
2011-01-05         NaN
2011-01-07    0.828404
2011-01-08         NaN
2011-01-10    3.300429
2011-01-12         NaN
dtype: float64

In [46]:
ts[::2]

2011-01-02    1.874317
2011-01-07    0.414202
2011-01-10    1.650215
dtype: float64

#### pandas的Numpy的datetime64数据类型以纳秒形式存储时间戳

In [47]:
ts.index.dtype

dtype('<M8[ns]')

#### DatetimeIndex中的各个标量值是pandas的Timestamp对象

In [48]:
stamp = ts.index[0]

In [49]:
stamp

Timestamp('2011-01-02 00:00:00')

#### 索引、选取、子集构造

#### TimeSeries是Series的一个子类，所以在索引以及数据选取方面行为都是一样的

In [50]:
stamp = ts.index[2]

In [51]:
stamp

Timestamp('2011-01-07 00:00:00')

In [53]:
ts[stamp]

0.41420205880234162

#### 更简单的办法：传入一个可以被解释为日期的字符串

In [54]:
ts['1/10/2011']

1.6502147098844078

In [55]:
ts['20110110']

1.6502147098844078

#### 对于较长的时间序列，只需要传入“年”或“年月”即可选取数据的切片

In [56]:
longer_ts = Series(np.random.randn(1000), index=pd.date_range('1/1/2000', periods=1000))

In [57]:
longer_ts

2000-01-01    0.483349
2000-01-02   -0.500831
2000-01-03    0.664282
2000-01-04   -0.141766
2000-01-05    0.130753
2000-01-06    1.311935
2000-01-07    0.535705
2000-01-08    0.407007
2000-01-09    0.769479
2000-01-10    1.781637
2000-01-11   -0.134653
2000-01-12    0.326779
2000-01-13    0.226018
2000-01-14   -0.263505
2000-01-15    1.467127
2000-01-16   -0.657822
2000-01-17   -0.431701
2000-01-18   -0.033751
2000-01-19    1.083123
2000-01-20   -0.101879
2000-01-21   -0.312029
2000-01-22   -0.737801
2000-01-23   -0.407252
2000-01-24   -0.665115
2000-01-25   -0.848637
2000-01-26   -1.130107
2000-01-27   -1.930013
2000-01-28   -2.021749
2000-01-29    0.376477
2000-01-30    1.247734
                ...   
2002-08-28   -0.335486
2002-08-29   -1.002669
2002-08-30    1.379136
2002-08-31   -2.086307
2002-09-01    0.507470
2002-09-02   -0.066370
2002-09-03    0.889719
2002-09-04    0.615194
2002-09-05   -2.622082
2002-09-06   -1.392356
2002-09-07    1.189186
2002-09-08    0.827333
2002-09-09 

In [58]:
longer_ts['2001']

2001-01-01   -1.022023
2001-01-02   -0.344996
2001-01-03    0.033343
2001-01-04    1.386902
2001-01-05    0.080076
2001-01-06    1.317936
2001-01-07    2.556896
2001-01-08    1.699479
2001-01-09    0.523952
2001-01-10   -0.050851
2001-01-11    0.702567
2001-01-12    1.628976
2001-01-13    0.160744
2001-01-14   -0.746844
2001-01-15   -0.213667
2001-01-16    0.864750
2001-01-17   -2.455367
2001-01-18   -0.392107
2001-01-19    1.026103
2001-01-20    1.211382
2001-01-21   -0.332744
2001-01-22    1.764798
2001-01-23    1.718743
2001-01-24   -0.958630
2001-01-25    0.816960
2001-01-26   -1.895595
2001-01-27    0.072862
2001-01-28   -1.055032
2001-01-29   -0.620038
2001-01-30   -0.444492
                ...   
2001-12-02    0.410426
2001-12-03   -1.115946
2001-12-04    0.519413
2001-12-05   -1.123911
2001-12-06   -0.246307
2001-12-07    1.647844
2001-12-08   -1.707982
2001-12-09    0.322704
2001-12-10   -1.415099
2001-12-11   -0.291932
2001-12-12   -0.607224
2001-12-13   -0.340985
2001-12-14 

In [60]:
longer_ts['2001-05']

2001-05-01   -1.789705
2001-05-02   -0.919251
2001-05-03   -0.552040
2001-05-04    0.099669
2001-05-05    1.075226
2001-05-06   -0.466862
2001-05-07    1.575866
2001-05-08   -1.206103
2001-05-09   -0.885263
2001-05-10    0.653853
2001-05-11   -1.876502
2001-05-12    0.534428
2001-05-13   -0.669728
2001-05-14    1.543802
2001-05-15    1.722032
2001-05-16    1.068207
2001-05-17    0.549499
2001-05-18    0.965051
2001-05-19    0.798242
2001-05-20    0.463578
2001-05-21   -0.065229
2001-05-22   -0.449178
2001-05-23   -0.901251
2001-05-24    0.274869
2001-05-25   -0.354923
2001-05-26   -2.898956
2001-05-27    2.966463
2001-05-28   -0.089493
2001-05-29   -1.738751
2001-05-30   -0.520603
2001-05-31    1.120712
Freq: D, dtype: float64

#### 通过日期进行切片的方式只对规则Series有效

In [61]:
ts[datetime(2011, 1, 7)]

0.41420205880234162

#### 由于大部分时间序列数据都是按照时间先后顺序排列，因此可以使用不存在于改时间序列中的时间戳对其进行切片，即查询范围

In [62]:
ts

2011-01-02    1.874317
2011-01-05   -0.148317
2011-01-07    0.414202
2011-01-08    0.015564
2011-01-10    1.650215
2011-01-12    0.248404
dtype: float64

In [65]:
ts['1/6/2011': '1/11/2011'] # 产生的是视图

2011-01-07    0.414202
2011-01-08    0.015564
2011-01-10    1.650215
dtype: float64

In [66]:
# 截取两个日期之间的TimeSeries

In [67]:
ts.truncate(after='1/9/2011')

2011-01-02    1.874317
2011-01-05   -0.148317
2011-01-07    0.414202
2011-01-08    0.015564
dtype: float64

#### 对DataFrame也有效，可以对DataFrame的行进行索引

In [68]:
dates = pd.date_range('1/1/2000', periods=100, freq='W-WED')

In [69]:
long_df = DataFrame(np.random.randn(100, 4),
                   index=dates,
                   columns=['Colorado', 'Texas', 'New York', 'Ohio'])

In [72]:
long_df.loc['2001-05']

,Colorado,Texas,New York,Ohio
2001-05-02,1.737010,-1.196329,-0.802822,-0.318167
2001-05-09,-0.344177,1.813739,-1.661357,-1.271154
2001-05-16,0.317846,1.003213,-0.939088,1.874071
2001-05-23,-1.377092,0.266191,0.221199,-1.353025
2001-05-30,-0.853960,-0.567092,-1.035977,0.138950


###  带有重复索引的时间序列

In [73]:
dates = pd.DatetimeIndex(['1/1/2000', '1/2/2000', '1/2/2000','1/2/2000','1/3/2000'])

In [74]:
dup_ts = Series(np.arange(5), index=dates)

In [75]:
dup_ts

2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int32

In [76]:
dup_ts.index.is_unique

False

In [77]:
# 对这个时间序列进行索引，如果不重复就会产生标量值，如果重复就会产生序列

In [79]:
dup_ts['1/3/2000'] # 不重复

4

In [82]:
dup_ts['1/2/2000']  # 重复

2000-01-02    1
2000-01-02    2
2000-01-02    3
dtype: int32

#### 相对具有非唯一时间戳的数据进行聚合，可以使用groupby，并传入level=0（索引的唯一一层）

In [84]:
grouped = dup_ts.groupby(level=0)

In [85]:
grouped.mean()

2000-01-01    0
2000-01-02    2
2000-01-03    4
dtype: int32

### 日期的范围，频率以及移动

#### 可以将之前的转换为一个具有固定频率（每日）的时间序列，只需要调用resample即可

In [86]:
ts

2011-01-02    1.874317
2011-01-05   -0.148317
2011-01-07    0.414202
2011-01-08    0.015564
2011-01-10    1.650215
2011-01-12    0.248404
dtype: float64

In [89]:
ts.resample('D')

DatetimeIndexResampler [freq=<Day>, axis=0, closed=left, label=left, convention=start, base=0]

### 生成日期范围

#### pandas.date_range可用于生成指定长度的DatetimeIndex

In [90]:
index = pd.date_range('4/1/2012', '6/1/2012')

In [91]:
index

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20',
               '2012-04-21', '2012-04-22', '2012-04-23', '2012-04-24',
               '2012-04-25', '2012-04-26', '2012-04-27', '2012-04-28',
               '2012-04-29', '2012-04-30', '2012-05-01', '2012-05-02',
               '2012-05-03', '2012-05-04', '2012-05-05', '2012-05-06',
               '2012-05-07', '2012-05-08', '2012-05-09', '2012-05-10',
               '2012-05-11', '2012-05-12', '2012-05-13', '2012-05-14',
               '2012-05-15', '2012-05-16', '2012-05-17', '2012-05-18',
               '2012-05-19', '2012-05-20', '2012-05-21', '2012-05-22',
               '2012-05-23', '2012-05-24', '2012-05-25', '2012-05-26',
      

#### 默认情况下，会产生按天计算的时间点。如果只传入起始或结束日期，那就还得传入一个表示一段时间的数字

In [92]:
pd.date_range(start='4/1/2012', periods=20)

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20'],
              dtype='datetime64[ns]', freq='D')

In [93]:
pd.date_range(end='6/1/2012', periods=20)

DatetimeIndex(['2012-05-13', '2012-05-14', '2012-05-15', '2012-05-16',
               '2012-05-17', '2012-05-18', '2012-05-19', '2012-05-20',
               '2012-05-21', '2012-05-22', '2012-05-23', '2012-05-24',
               '2012-05-25', '2012-05-26', '2012-05-27', '2012-05-28',
               '2012-05-29', '2012-05-30', '2012-05-31', '2012-06-01'],
              dtype='datetime64[ns]', freq='D')

#### 如果想生成一个由每月最后一个工作日组成的日期索引，可以传入“BM”频率（表示business end of month），这样就只会包含时间间隔内（或刚好在边界上）符合频率要求的日期

In [94]:
pd.date_range('1/1/2000','12/1/2000', freq='BM')

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-28',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-29', '2000-10-31', '2000-11-30'],
              dtype='datetime64[ns]', freq='BM')

#### date_range默认会保留起始和结束时间戳的时间信息（如果有的话）：

In [95]:
pd.date_range('5/2/2012 12:56:31', periods=5)

DatetimeIndex(['2012-05-02 12:56:31', '2012-05-03 12:56:31',
               '2012-05-04 12:56:31', '2012-05-05 12:56:31',
               '2012-05-06 12:56:31'],
              dtype='datetime64[ns]', freq='D')

#### 虽然起始和结束日期带有时间信息，但希望产生一组被规范化（normalize）到午夜的时间戳。normalize选项可以实现

In [96]:
pd.date_range('5/2/2012 12:56:31', periods=5, normalize=True)

DatetimeIndex(['2012-05-02', '2012-05-03', '2012-05-04', '2012-05-05',
               '2012-05-06'],
              dtype='datetime64[ns]', freq='D')

### 频率和日期偏移量

#### pandas中的频率是由一个基础频率和一个乘数组成的。基础频率通常是以一个字符串别名表示，如“M”表示每月，“H”表示每小时
对于每个基础频率，都有一个呗成为日期偏移量（date offset）的对象与之对应。
例如，按小时计算的频率可以用Hour类表示

In [97]:
from pandas.tseries.offsets import Hour, Minute

In [98]:
hour = Hour()

In [99]:
hour

<Hour>

In [100]:
# 传入一个整数即可定义偏移量的倍数
four_hours = Hour(4)

In [101]:
four_hours

<4 * Hours>

#### 无需显示创建，使用“H”或“4H”这样的字符串别名即可

In [102]:
pd.date_range('1/1/2000', '1/3/2000 23:59', freq='4h')

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 04:00:00',
               '2000-01-01 08:00:00', '2000-01-01 12:00:00',
               '2000-01-01 16:00:00', '2000-01-01 20:00:00',
               '2000-01-02 00:00:00', '2000-01-02 04:00:00',
               '2000-01-02 08:00:00', '2000-01-02 12:00:00',
               '2000-01-02 16:00:00', '2000-01-02 20:00:00',
               '2000-01-03 00:00:00', '2000-01-03 04:00:00',
               '2000-01-03 08:00:00', '2000-01-03 12:00:00',
               '2000-01-03 16:00:00', '2000-01-03 20:00:00'],
              dtype='datetime64[ns]', freq='4H')

#### 大部分偏移量对象可以通过加法进行连接：

In [103]:
Hour(2) + Minute(30)

<150 * Minutes>

In [105]:
pd.date_range('1/1/2000', periods=10, freq='1h30min')

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 01:30:00',
               '2000-01-01 03:00:00', '2000-01-01 04:30:00',
               '2000-01-01 06:00:00', '2000-01-01 07:30:00',
               '2000-01-01 09:00:00', '2000-01-01 10:30:00',
               '2000-01-01 12:00:00', '2000-01-01 13:30:00'],
              dtype='datetime64[ns]', freq='90T')

#### WOM日期

#### WOM（Week Of Month）是一种非常实用的频率类。它以WOM开头，使你能获得诸如“每月第三个星期五”之类的日期

In [106]:
rng = pd.date_range('1/1/2012', '9/1/2012', freq='WOM-3FRI')

In [108]:
list(rng)

[Timestamp('2012-01-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-02-17 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-03-16 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-04-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-05-18 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-06-15 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-07-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-08-17 00:00:00', freq='WOM-3FRI')]

### 移动（超前和滞后）数据

#### 移动（shifting）指的是沿着时间轴将数据前移或后移。Series和DataFrame都有shift方法用于执行单纯的前后移动，保持索引不变

In [109]:
ts = Series(np.random.randn(4),
           index=pd.date_range('1/1/2000', periods=4, freq='M'))

In [110]:
ts

2000-01-31   -0.566106
2000-02-29    0.224816
2000-03-31   -0.161165
2000-04-30    0.495453
Freq: M, dtype: float64

In [111]:
ts.shift(2)  # 从上往下平移2个

2000-01-31         NaN
2000-02-29         NaN
2000-03-31   -0.566106
2000-04-30    0.224816
Freq: M, dtype: float64

In [113]:
ts.shift(-2) # 从下往上平移2个

2000-01-31   -0.161165
2000-02-29    0.495453
2000-03-31         NaN
2000-04-30         NaN
Freq: M, dtype: float64

#### shift通常用于计算一个时间序列或多个时间序列中的百分比变化

In [114]:
ts.shift(2, freq='M')

2000-03-31   -0.566106
2000-04-30    0.224816
2000-05-31   -0.161165
2000-06-30    0.495453
Freq: M, dtype: float64

In [115]:
ts.shift(3, freq='D')

2000-02-03   -0.566106
2000-03-03    0.224816
2000-04-03   -0.161165
2000-05-03    0.495453
dtype: float64

In [116]:
ts.shift(1, freq='3D')

2000-02-03   -0.566106
2000-03-03    0.224816
2000-04-03   -0.161165
2000-05-03    0.495453
dtype: float64

In [117]:
ts.shift(1, freq='90T')

2000-01-31 01:30:00   -0.566106
2000-02-29 01:30:00    0.224816
2000-03-31 01:30:00   -0.161165
2000-04-30 01:30:00    0.495453
Freq: M, dtype: float64

#### 通过偏移量对日期进行位移

In [118]:
# pandas的日期偏移量还可以用在datetime或Timestamp对象上

In [119]:
from pandas.tseries.offsets import Day, MonthEnd

In [120]:
now = datetime(2011, 11, 7)

In [124]:
now + 3 * Day()

<6 * Days>

In [127]:
offset = MonthEnd()

In [132]:
ts = Series(np.random.randn(20), index=pd.date_range('1/15/2000', periods=20, freq='4d'))

In [133]:
ts.groupby(offset.rollforward).mean()

2000-01-31   -0.067585
2000-02-29   -0.167265
2000-03-31   -0.489632
dtype: float64

In [134]:
ts.resample('M', how='mean')

E:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  if __name__ == '__main__':


2000-01-31   -0.067585
2000-02-29   -0.167265
2000-03-31   -0.489632
Freq: M, dtype: float64

### 时区处理

#### pytz时区信息数据库

In [135]:
import pytz

In [136]:
pytz.common_timezones[-5:]

['US/Eastern', 'US/Hawaii', 'US/Mountain', 'US/Pacific', 'UTC']

In [137]:
# 要从pytz中获取时区对象，使用pytz,timezone即可
tz = pytz.timezone('US/Eastern')

In [138]:
tz

<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>

### 本地化和转换

#### 默认情况下，pandas中的时间序列是单纯的（navie）时区

In [139]:
rng = pd.date_range('3/9/2012 9:30', periods=6, freq='D')

In [140]:
rng

DatetimeIndex(['2012-03-09 09:30:00', '2012-03-10 09:30:00',
               '2012-03-11 09:30:00', '2012-03-12 09:30:00',
               '2012-03-13 09:30:00', '2012-03-14 09:30:00'],
              dtype='datetime64[ns]', freq='D')

In [141]:
ts = Series(np.random.randn(len(rng)), index=rng)

In [142]:
ts

2012-03-09 09:30:00    0.655548
2012-03-10 09:30:00    1.441447
2012-03-11 09:30:00    0.474472
2012-03-12 09:30:00   -0.439803
2012-03-13 09:30:00    0.888993
2012-03-14 09:30:00   -1.562403
Freq: D, dtype: float64

In [144]:
# 其索引的tz字段为None
print ts.index.tz

None


#### 在生成日期范围的时候还可以加上一个时区集

In [145]:
pd.date_range('3/9/2012 9:30', periods=10, freq='D', tz='UTC')

DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00', '2012-03-16 09:30:00+00:00',
               '2012-03-17 09:30:00+00:00', '2012-03-18 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

#### 从单纯到本地化的转换时通过tz_localize方法处理的

In [148]:
ts_utc = ts.tz_localize('UTC')

In [149]:
ts_utc

2012-03-09 09:30:00+00:00    0.655548
2012-03-10 09:30:00+00:00    1.441447
2012-03-11 09:30:00+00:00    0.474472
2012-03-12 09:30:00+00:00   -0.439803
2012-03-13 09:30:00+00:00    0.888993
2012-03-14 09:30:00+00:00   -1.562403
Freq: D, dtype: float64

In [151]:
list(ts_utc.index)

[Timestamp('2012-03-09 09:30:00+0000', tz='UTC', freq='D'),
 Timestamp('2012-03-10 09:30:00+0000', tz='UTC', freq='D'),
 Timestamp('2012-03-11 09:30:00+0000', tz='UTC', freq='D'),
 Timestamp('2012-03-12 09:30:00+0000', tz='UTC', freq='D'),
 Timestamp('2012-03-13 09:30:00+0000', tz='UTC', freq='D'),
 Timestamp('2012-03-14 09:30:00+0000', tz='UTC', freq='D')]

#### 一旦时间序列被本地化到某个特定时区，就可以用tz_convert将其换转到别的时区了

In [153]:
ts_utc.tz_convert('US/Eastern')

2012-03-09 04:30:00-05:00    0.655548
2012-03-10 04:30:00-05:00    1.441447
2012-03-11 05:30:00-04:00    0.474472
2012-03-12 05:30:00-04:00   -0.439803
2012-03-13 05:30:00-04:00    0.888993
2012-03-14 05:30:00-04:00   -1.562403
Freq: D, dtype: float64

#### 对于上面这种时间序列（它跨越了美国东部时区的夏令时转变期），我们可以将其本地化到EST，然后转换为UTC或柏林时间

In [154]:
ts_eastern = ts.tz_localize('US/Eastern')

In [155]:
ts_eastern.tz_convert('UTC')

2012-03-09 14:30:00+00:00    0.655548
2012-03-10 14:30:00+00:00    1.441447
2012-03-11 13:30:00+00:00    0.474472
2012-03-12 13:30:00+00:00   -0.439803
2012-03-13 13:30:00+00:00    0.888993
2012-03-14 13:30:00+00:00   -1.562403
Freq: D, dtype: float64

In [156]:
ts_eastern.tz_convert('Europe/Berlin')

2012-03-09 15:30:00+01:00    0.655548
2012-03-10 15:30:00+01:00    1.441447
2012-03-11 14:30:00+01:00    0.474472
2012-03-12 14:30:00+01:00   -0.439803
2012-03-13 14:30:00+01:00    0.888993
2012-03-14 14:30:00+01:00   -1.562403
Freq: D, dtype: float64

In [157]:
ts.index.tz_localize('Asia/Shanghai')

DatetimeIndex(['2012-03-09 09:30:00+08:00', '2012-03-10 09:30:00+08:00',
               '2012-03-11 09:30:00+08:00', '2012-03-12 09:30:00+08:00',
               '2012-03-13 09:30:00+08:00', '2012-03-14 09:30:00+08:00'],
              dtype='datetime64[ns, Asia/Shanghai]', freq='D')

### 操作时区意识型Timestamp对象

#### Timestamp对象也能被从单纯性本地化为时区意识型，并从一个时区转换到另一个时区

In [158]:
stamp = pd.Timestamp('2011-03-12 04:00')

In [159]:
stamp

Timestamp('2011-03-12 04:00:00')

In [160]:
stamp_utc = stamp.tz_localize('utc')

In [161]:
stamp_utc

Timestamp('2011-03-12 04:00:00+0000', tz='UTC')

In [163]:
stamp_utc.tz_convert('US/Eastern')

Timestamp('2011-03-11 23:00:00-0500', tz='US/Eastern')

In [165]:
# 在创建Timestamp时，还可以传入一个时区信息
stamp_moscow = pd.Timestamp('2011-03-12 4:00', tz='Europe/Moscow')

In [166]:
stamp_moscow

Timestamp('2011-03-12 04:00:00+0300', tz='Europe/Moscow')

In [169]:
stamp_utc.value  # 时区意识型Timestamp对象在内部保存了一个UTC时间戳值（自1970年1月1日）算起的纳秒数。而且这个UTC值在时区转换过程中不会变化

1299902400000000000L

In [170]:
stamp_utc.tz_convert('US/Eastern').value

1299902400000000000L

In [171]:
# 夏令时转变提前30分钟
from pandas.tseries.offsets import Hour

In [172]:
stamp = pd.Timestamp('2012-03-12 01:30', tz='US/Eastern')

In [173]:
stamp

Timestamp('2012-03-12 01:30:00-0400', tz='US/Eastern')

In [174]:
stamp + Hour()

Timestamp('2012-03-12 02:30:00-0400', tz='US/Eastern')

In [175]:
# 夏令时转变提前90分钟
stamp = pd.Timestamp('2012-11-04 00:30', tz='US/Eastern')

In [177]:
stamp

Timestamp('2012-11-04 00:30:00-0400', tz='US/Eastern')

In [178]:
stamp + 2* Hour()

Timestamp('2012-11-04 01:30:00-0500', tz='US/Eastern')

#### 不同时区之间的运算

In [6]:
from pandas import DataFrame, Series
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


E:\Anaconda\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


#### 如果两个时间序列的时区不同，在将他们合并到一起时，最终结果就会使UTC。由于时间戳其实是以UTC存储的，所以不需要任何转换

In [4]:
rng = pd.date_range('3/7/2012 9:30', periods=10, freq='B')

In [7]:
ts = Series(np.random.randn(len(rng)), index=rng)

In [8]:
ts

2012-03-07 09:30:00    0.102418
2012-03-08 09:30:00    1.103018
2012-03-09 09:30:00   -0.420620
2012-03-12 09:30:00   -0.637973
2012-03-13 09:30:00   -1.929466
2012-03-14 09:30:00   -1.694004
2012-03-15 09:30:00   -0.277746
2012-03-16 09:30:00    0.377937
2012-03-19 09:30:00    1.312346
2012-03-20 09:30:00    0.838627
Freq: B, dtype: float64

In [9]:
ts1 = ts[:7].tz_localize('Europe/London')

In [10]:
ts1

2012-03-07 09:30:00+00:00    0.102418
2012-03-08 09:30:00+00:00    1.103018
2012-03-09 09:30:00+00:00   -0.420620
2012-03-12 09:30:00+00:00   -0.637973
2012-03-13 09:30:00+00:00   -1.929466
2012-03-14 09:30:00+00:00   -1.694004
2012-03-15 09:30:00+00:00   -0.277746
Freq: B, dtype: float64

In [15]:
ts2 = ts1[2:].tz_convert('Europe/Moscow')
ts2

2012-03-09 13:30:00+04:00   -0.420620
2012-03-12 13:30:00+04:00   -0.637973
2012-03-13 13:30:00+04:00   -1.929466
2012-03-14 13:30:00+04:00   -1.694004
2012-03-15 13:30:00+04:00   -0.277746
Freq: B, dtype: float64

In [16]:
result = ts1 + ts2

In [13]:
result

2012-03-07 09:30:00+00:00         NaN
2012-03-08 09:30:00+00:00         NaN
2012-03-09 09:30:00+00:00   -0.841239
2012-03-12 09:30:00+00:00   -1.275945
2012-03-13 09:30:00+00:00   -3.858933
2012-03-14 09:30:00+00:00   -3.388008
2012-03-15 09:30:00+00:00   -0.555493
Freq: B, dtype: float64

In [14]:
result.index

DatetimeIndex(['2012-03-07 09:30:00+00:00', '2012-03-08 09:30:00+00:00',
               '2012-03-09 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='B')

#### 时期及其算术运算

In [17]:
p = pd.Period(2007, freq='A-DEC')

In [18]:
p

Period('2007', 'A-DEC')

#### 这个Period对象表示的是从2007年1月1日到2007年12月31日之间的整段时间
只需要对Period对象加上或减去一个整数即可达到根据其频率进行位移的效果

In [19]:
p+5

Period('2012', 'A-DEC')

In [20]:
p-2

Period('2005', 'A-DEC')

#### 如果两个Period对象拥有相同的频率，则他们的差就是他们之间的单位数量

In [21]:
pp = pd.Period('2014', freq='A-DEC')

In [22]:
pp

Period('2014', 'A-DEC')

In [23]:
pp - p

7L

#### period_range函数可用于创建规则的时期范围：

In [24]:
rng = pd.period_range('1/1/2000', '6/30/2000',freq='M')

In [25]:
rng

PeriodIndex(['2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06'], dtype='period[M]', freq='M')

#### PeriodIndex类保存了一组Period，它可以在任何pandas数据结构中被用作轴索引

In [26]:
Series(np.random.randn(6), index=rng)

2000-01    0.633137
2000-02   -0.412950
2000-03   -1.151824
2000-04    1.461933
2000-05   -1.015274
2000-06    0.323844
Freq: M, dtype: float64

#### PeriodIndex类的构造函数还允许直接使用一组字符串

In [27]:
values = ['2001Q3', '2002Q2', '2003Q1']

In [28]:
index = pd.PeriodIndex(values, freq='Q-DEC')

In [29]:
index

PeriodIndex(['2001Q3', '2002Q2', '2003Q1'], dtype='period[Q-DEC]', freq='Q-DEC')

#### 时期的频率转换

In [30]:
p = pd.Period('2007', freq='A-DEC')

In [31]:
p.asfreq('M', how='start')

Period('2007-01', 'M')

In [36]:
p.asfreq('M', how='end')

Period('2007-12', 'M')

In [37]:
p = pd.Period('2007', freq='A-JUN')

In [38]:
p.asfreq('M','start')

Period('2006-07', 'M')

In [39]:
p.asfreq('M','end')

Period('2007-06', 'M')

In [40]:
p = pd.Period('2007-08', 'M')

In [41]:
p.asfreq('A-JUN')

Period('2008', 'A-JUN')

In [42]:
rng = pd.period_range('2006', '2009', freq='A-DEC')

In [43]:
ts = Series(np.random.randn(len(rng)), index=rng)

In [44]:
ts

2006   -1.133531
2007    0.668587
2008    0.289095
2009    0.556943
Freq: A-DEC, dtype: float64

In [45]:
ts.asfreq('M', how='start')

2006-01   -1.133531
2007-01    0.668587
2008-01    0.289095
2009-01    0.556943
Freq: M, dtype: float64

In [47]:
ts.asfreq('B', how='end')

2006-12-29   -1.133531
2007-12-31    0.668587
2008-12-31    0.289095
2009-12-31    0.556943
Freq: B, dtype: float64

#### 按季度计算的时期频率

In [48]:
p = pd.Period('2014Q4', freq='Q-JAN')

In [49]:
p

Period('2014Q4', 'Q-JAN')

#### 在以1月结束的财年中，2014Q4是从11月到1月

In [50]:
p.asfreq('D', 'start')

Period('2013-11-01', 'D')

In [51]:
p.asfreq('D', 'end')

Period('2014-01-31', 'D')

#### 要获取该季度倒数第二个工作日下午4点的时间戳

In [52]:
p4pm = (p.asfreq('B', 'e') - 1).asfreq('T', 's') + 16 * 60

In [53]:
p4pm

Period('2014-01-30 16:00', 'T')

In [54]:
p4pm.to_timestamp()

Timestamp('2014-01-30 16:00:00')

#### period_range还可以用于生产季度型范围

In [59]:
rng = pd.period_range('2011Q3', '2012Q4', freq='Q-JAN')

In [60]:
rng

PeriodIndex(['2011Q3', '2011Q4', '2012Q1', '2012Q2', '2012Q3', '2012Q4'], dtype='period[Q-JAN]', freq='Q-JAN')

In [61]:
ts= Series(np.arange(len(rng)), index=rng)

In [62]:
ts

2011Q3    0
2011Q4    1
2012Q1    2
2012Q2    3
2012Q3    4
2012Q4    5
Freq: Q-JAN, dtype: int32

In [64]:
new_rng = (rng.asfreq('B', 'e') -1).asfreq('T', 's') + 16*60

In [65]:
new_rng

PeriodIndex(['2010-10-28 16:00', '2011-01-28 16:00', '2011-04-28 16:00',
             '2011-07-28 16:00', '2011-10-28 16:00', '2012-01-30 16:00'],
            dtype='period[T]', freq='T')

In [66]:
ts_index = new_rng.to_timestamp()

In [67]:
ts_index

DatetimeIndex(['2010-10-28 16:00:00', '2011-01-28 16:00:00',
               '2011-04-28 16:00:00', '2011-07-28 16:00:00',
               '2011-10-28 16:00:00', '2012-01-30 16:00:00'],
              dtype='datetime64[ns]', freq=None)